In [1]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

In [2]:
import numpy as np
# Users
U = list(range(0, 8))

# Servers
S = list(range(0, 3))

# D = {CPU, RAM, storage, bandwidth}
CPU = 1
RAM = 2
storage = 3
BW = 4
D = [CPU, RAM, storage, BW]

# Users u has memory requirements = w_j
w = [[CPU for j in U],[RAM for j in U],[storage for j in U],[BW for j in U]]

# Each server has memory capacity C
C= [[CPU for i in S],[RAM for i in S],[storage for i in S],[BW for i in S]]

In [3]:
########## DECLARE THE MIP Solver ##########
solver = pywraplp.Solver.CreateSolver('SCIP')

In [4]:
########## DEFINE THE VARIABLES ##########
# x_i,j = True if user u_j is allocated to edge server s_i
# x_i,j = False otherwise
x = {}
for j in U:
    for i in S:
        x[i, j] = solver.BoolVar(f'c[{i}][{j}]')
        #print(x[i,j])

# y_i = True if edge server s_i is used to serve users
# y_i = False otherwise
y = {}
for i in S:
    y[i] = solver.BoolVar(f'c[{i}]')

print('Number of variables =', solver.NumVariables())

Number of variables = 27


In [5]:
########## DEFINE THE CONSTRAINTS ##########

#Proximity constraint
#for j in U:
    #for i in S:
        # Missing coverage condition: 
        #solver.Add(
            #x[i, j]==0
        #)

# For each server i
# the sum of memory requirements of the users allocated to server i
# does not exceed the memory capacity of i
for i in range(len(S)):
    for k in range(len(D)):
        solver.Add(
            solver.Sum([
                x[i, j] * w[k][j] for j in range(len(U))
            ]) <= C[k][i]*y[i]
        )

# Each user can be allocated just once
for j in U:
    solver.Add(
        solver.Sum([x[i, j] for i in S]) <= 1
    )

# Constraint 3: x, y ≥ 0
# has already been included in the definition of the variables

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 20


In [10]:
########## DEFINE THE OBJECTIVE ##########

# Maximize the number of allocated users
objective = solver.Objective()
for j in U:
    for i in S:
        objective.SetCoefficient(x[i, j], 1)
objective.SetMaximization()

# Minimize the number of required edge servers
#objectiveMin = solver.Objective()

#for i in S:
 #   objectiveMin.SetCoefficient(y[i], 1)
#objectiveMin.SetMinimization()

In [11]:
########## CALL THE SOLVER ##########
status = solver.Solve()


########### DISPLAY THE SOLUTION ##########
if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    for j in U:
        for i in S:
            if int(x[i,j].solution_value()) == 1:
                print(f'User {j} has been allocated on node {i}')
else:
    print('The problem does not have an optimal solution.')

Solution:
Objective value = 6.0
User 0 has been allocated on node 0
User 1 has been allocated on node 1
User 2 has been allocated on node 2
